<a href="https://colab.research.google.com/github/lauren-safwat/World-University-Rankings-Dashboard/blob/main/World_University_Rankings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1624]:
# !pip3 install dash
# !pip3 install jupyter_dash
# !pip3 install dash_bootstrap_components

In [1625]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

import jupyter_dash
from dash import Dash, html, dcc, no_update
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
from dash.exceptions import PreventUpdate
import plotly.express as px
import plotly.graph_objs as go
from datetime import date
from datetime import datetime

In [1626]:
unis = pd.read_csv('World_University_Rankings.csv')

In [1627]:
# unis[:100]

In [1628]:
unis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1373 entries, 0 to 1372
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   University              1373 non-null   object 
 1   Country                 1373 non-null   object 
 2   City                    1322 non-null   object 
 3   Region                  1373 non-null   object 
 4   Logo                    1373 non-null   object 
 5   Link                    1373 non-null   object 
 6   Type                    1373 non-null   object 
 7   Size                    1373 non-null   object 
 8   Faculty_count           1373 non-null   int64  
 9   International_Students  1373 non-null   int64  
 10  Students_count          1373 non-null   int64  
 11  Rank_2017               1373 non-null   int64  
 12  Rank_2018               1373 non-null   int64  
 13  Rank_2019               1373 non-null   int64  
 14  Rank_2020               1373 non-null   

In [1629]:
year=2022
x=np.arange(2017,2023)
univ = unis.loc[0,'University']
print(univ)
y=unis[unis['University']=='Assiut University'].iloc[:,11:17].values.flatten().tolist()
x = [x[i] for i in range(len(y)) if y[i]>0]
y = [y[i] for i in range(len(y)) if y[i]>0]

print(unis[unis['Region'].isin( ['Africa','Asia'])])


fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y, name="linear",
                    line_shape='linear'))
fig.add_trace(go.Scatter(x=[x[y.index(np.min(y))]], y=[np.min(y)], mode = 'markers',
                         marker_symbol = 'star',
                         marker_size = 15))
y=unis[unis['University']=='The American University in Cairo'].iloc[:,11:17].values.flatten().tolist()
# fig.add_trace(go.Scatter(x=x, y=y, name="linear",
#                     line_shape='linear'))
# fig.add_trace(go.Scatter(x=[x[y.index(np.min(y))]], y=[np.min(y)], mode = 'markers',
#                          marker_symbol = 'star',
#                          marker_size = 15))
fig['layout']['yaxis']['autorange'] = "reversed"
fig.show()

University Paris-Saclay
                         University     Country      City  Region  \
1       Baku Engineering University  Azerbaijan      Baku    Asia   
6           Arabian Gulf University     Bahrain    Manama    Asia   
7                 Khazar University  Azerbaijan      Baku    Asia   
18    Al Akhawayn University Ifrane     Morocco   Ifranne  Africa   
19            Ochanomizu University       Japan     Tokyo    Asia   
...                             ...         ...       ...     ...   
1368            Université de Tunis     Tunisia     Tunis  Africa   
1369   Université de Tunis El Manar     Tunisia     Tunis  Africa   
1370             Yarmouk University      Jordan     Irbid    Asia   
1371    Yildiz Technical University      Turkey  Istanbul    Asia   
1372             Zagazig University       Egypt   Zagazig  Africa   

                                                   Logo  \
1     https://www.topuniversities.com/sites/default/...   
6     https://www.topuniversi

In [1630]:
app = jupyter_dash.JupyterDash(external_stylesheets=[dbc.themes.BOOTSTRAP])

In [1631]:
# x = np.logical_and(unis['Country']=='Russia', unis['Rank_2022']>0)
# fig = px.scatter_mapbox(
#         unis[x].iloc[:100,:], lon='Longitude', lat='Latitude', color='Rank_2022', hover_name='University', size_max=40, hover_data=['Country'], zoom=3)
# fig.update_layout(mapbox_style="carto-positron")
# fig.show()

In [1632]:
app.layout = dbc.Container(children=[
    dbc.Row([
        dbc.Col(dcc.Graph(id='map', figure={}), width=8),
        dbc.Col(html.Div(id='table', style={'maxHeight': '350px', 'overflowY': 'scroll'}))
    ]),

    dbc.Row([
        dbc.Col(dcc.Slider(id='mapSlider',
                  min=2017,
                  max=2022,
                  value=2022,
                  step=None,
                  marks={i: str(i) for i in range(2017, 2023)}
                  ), width=6),
             
        dbc.Col(dcc.Dropdown([str(region) for region in unis['Region'].unique()], placeholder='Select Region', id='map_region'), width=3),
        dbc.Col(dcc.Dropdown(placeholder='Select Country', id='map_country')),
    ]),
    

    dbc.Row([
        dbc.Col(
            [dbc.Row(dcc.Graph(id='bar_uniCount', figure={})),     
            dbc.Row(dbc.Col(dcc.Dropdown([str(region) for region in unis['Region'].unique()], placeholder='Select Region', id='bar_region_uniCount'),width=4))],
            width=6
        ),

        dbc.Col(
            [dbc.Row(dcc.Graph(id='bar_intCount', figure={})),     
            dbc.Row(dbc.Col(dcc.Dropdown([str(region) for region in unis['Region'].unique()], placeholder='Select Region', id='bar_region_intCount'),width=4))],
            width=6
        )
    ]),
    dbc.Row([

        dbc.Col(
            [dbc.Row(dcc.Graph(id='pie_type', figure={})),     
            dbc.Row([dbc.Col(dcc.Dropdown([str(region) for region in unis['Region'].unique()], placeholder='Select Region', id='pie_region_type'),width=4),
                     dbc.Col(dcc.Dropdown(placeholder='Select Country', id='pie_country_type'),width=4)
                     ])],
            width=6
        )
    ]),
   
   html.H1('Second Page Content'),
    
    dbc.Row([
             dbc.Col(
            dbc.Row(dcc.Graph(id='line_uni', figure={})),     
            width=6
        ),
        dbc.Col([dbc.Col(dcc.Dropdown([str(region) for region in unis['Region'].unique()], placeholder='Select Region',multi=True, id='line_region_uni'),width=4),
                          dbc.Col(dcc.Dropdown(placeholder='Select Country',multi=True, id='line_country_uni'),width=4),
                          dbc.Col(dcc.Dropdown(placeholder='Select University',multi=True, id='line_uni_uni'),width=7),
                          ])
    ])   
   

    ################ SECOND PAGE GRAPHS ###############
    
    
    


])

In [1633]:
@app.callback(
    Output('map_country', 'options'),
    Output('map', 'figure'),
    Output("table", "children"),
    Input('mapSlider', 'value'),
    Input('map_region', 'value'),
    Input('map_country', 'value')
)


def updateMap(year, region, country):
    year = 'Rank_' + str(year)
    df = unis[unis[year] > 0].sort_values(year, axis=0)
    print
    zoom = 0.6

    if region:
        zoom = 1.8
        df = df[df['Region'] == region]

    if country:
        zoom = 3
        df = df[df['Country'] == country]

    # token = 'pk.eyJ1IjoiYWZuYW5ldG1hbiIsImEiOiJjbDJqN3F5cW0wMHprM2VwazQ0aHdvZDBhIn0.X032SoU1tft9zgusUaEqSQ'
    # fig = go.Figure(go.Scattermapbox(lon=df[:100]['Longitude'],
    #                                  lat=df[:100]['Latitude'],
    #                                  mode='markers',
    #                                  marker = {'size': 10, 'symbol': 'college'},
    #                                  ))
    # fig.update_layout(mapbox = {'accesstoken': token, 'style': "outdoors", 'zoom': zoom})

    # px.set_mapbox_access_token('pk.eyJ1IjoiYWZuYW5ldG1hbiIsImEiOiJjbDJqN3F5cW0wMHprM2VwazQ0aHdvZDBhIn0.X032SoU1tft9zgusUaEqSQ')
    fig = px.scatter_mapbox(df[:100],
                            lon='Longitude',
                            lat='Latitude',
                            color=year,
                            hover_name='University',
                            hover_data=['Country'],
                            zoom=zoom,
                            title = 'Ranking of Universities Around the World',
                            mapbox_style='carto-positron')
    
    fig.update_traces(marker = {'size':10, 'opacity':0.5}, selector={'type': 'scattermapbox'})

    countries = [{'label': str(i), 'value':str(i)} for i in df['Country'].unique()]

    table = dbc.Table.from_dataframe(df[[year, 'University']], striped=True, bordered=True, hover=True, responsive=True)
    
    return countries, fig, table

In [1634]:
@app.callback(
    Output('bar_uniCount', 'figure'),
    Input('bar_region_uniCount', 'value'),
)

def updateBar(region):

    #countries = [{'label': str(i), 'value':str(i)} for i in unis['Country'].unique()]
    x=unis.groupby(['Region'])['University'].count()
    y=x.keys()
        

    if region:
        df = unis[unis['Region'] == region]
        #countries = [{'label': str(i), 'value':str(i)} for i in df['Country'].unique()]
        x=df.groupby(['Country'])['University'].count()
        y=x.keys()
    print(sorted(zip(x,y)))
    x = [i[0] for i in sorted(zip(x,y))]
    y = [i[1] for i in sorted(zip(x,y))]
        
    fig = go.Figure(go.Bar(
                x=list(x),
                y=list(y),
                orientation='h',
                text=x,
                marker={
                'color': x,
                'colorscale': 'bluyl'
                }))
    fig.update_layout(title="University Count",
                      barmode='group',
                      bargap=0.0,
                      bargroupgap=0.0
                     )
    return fig

In [1635]:
@app.callback(
    Output('bar_intCount', 'figure'),
    Input('bar_region_intCount', 'value'),
)

def updateBar(region):

    #countries = [{'label': str(i), 'value':str(i)} for i in unis['Country'].unique()]
    x=unis.groupby(['Region'])['International_Students'].sum()
    y=x.keys()
        

    if region:
        df = unis[unis['Region'] == region]
        #countries = [{'label': str(i), 'value':str(i)} for i in df['Country'].unique()]
        x=df.groupby(['Country'])['International_Students'].sum()
        y=x.keys()
    print(sorted(zip(x,y)))
    x = [i[0] for i in sorted(zip(x,y))]
    y = [i[1] for i in sorted(zip(x,y))]
    
    fig = go.Figure(go.Bar(
                x=list(x),
                y=list(y),
                orientation='h',
                text=x,
                marker={
                'color': x,
                'colorscale': 'bluyl'
                }))
    fig.update_layout(title="Number of International Students",
                      barmode='group',
                      bargap=0.0,
                      bargroupgap=0.0)
    return fig

In [1636]:
@app.callback(
    Output('pie_country_type', 'options'),
    Output('pie_type', 'figure'),
    Input('pie_region_type', 'value'),
    Input('pie_country_type', 'value')
)

def updateBar(region,country):
    df=unis
    labels=['Private','Public']        

    if region:
        df = unis[unis['Region'] == region]
    if country :
      df = df[df['Country'] == country]
    
    values=[df[df['Type']=='Private'].count()['University'],df[df['Type']=='Public'].count()['University']]
    countries = [{'label': str(i), 'value':str(i)} for i in df['Country'].unique()]

    fig = go.Figure(data=[go.Pie(labels=labels, values=values,
                             text=values,
                             marker = dict(colors = ['pink','blue'])
                             )])
    fig.update_layout(title="Public Vs Private Universites ")
    return countries,fig


In [1637]:
@app.callback(
    Output('line_uni_uni', 'options'),
    Output('line_country_uni', 'options'),
    Output('line_uni', 'figure'),
    Input('line_region_uni', 'value'),
    Input('line_country_uni', 'value'),
    Input('line_uni_uni', 'value')
)

def updateLine(region,country,university):
  df = unis
  
  x=np.arange(2017,2023)
  if region :
    df = unis[unis['Region'].isin(region) ]
  if country :
    df = df[df['Country'].isin(country)]
  if not university:
    university = ['Cairo University']
  fig = go.Figure()
  for uni in university:
    y=df[df['University']==uni].iloc[:,11:17].values.flatten().tolist()
    x = [x[i] for i in range(len(y)) if y[i]>0]
    y = [y[i] for i in range(len(y)) if y[i]>0]
    
    fig.add_trace(go.Scatter(x=x, y=y, name="linear",
                        line_shape='linear'))
    fig.add_trace(go.Scatter(x=[x[y.index(np.min(y))]], y=[np.min(y)], mode = 'markers',
                            marker_symbol = 'circle',
                            marker_size = 15))
  fig['layout']['yaxis']['autorange'] = "reversed"
  fig['layout']['xaxis']['dtick'] = 1
  # fig.update_layout(
  #   xaxis = dict(
  #       tickmode = 'linear',
  #       dtick = 1
  #   ))
  countries = [{'label': str(i), 'value':str(i)} for i in df['Country'].unique()]
  universities = [{'label': str(i), 'value':str(i)} for i in df['University'].unique()]
  return universities, countries, fig



In [1638]:
app.run_server()

Dash app running on:


<IPython.core.display.Javascript object>

In [1639]:
# from jupyter_dash import JupyterDash
# from dash import Dash, dcc, html, Input, Output, no_update
# import plotly.graph_objects as go
# import pandas as pd

# ## create sample random data
# df = pd.DataFrame({
#     'x': [80,2,10],
#     'y': [20,3,25],
#     'color': ['red','green','blue'],
#     'img_url': [
#         "https://upload.wikimedia.org/wikipedia/commons/0/03/Refugee_care_near_Poland_border_train_station_20220228.jpg",
#         "https://upload.wikimedia.org/wikipedia/commons/4/45/Ukrainian_refugees_from_2022%2C_crossing_into_Poland.jpg",
#         "https://upload.wikimedia.org/wikipedia/commons/b/b2/Warsaw_Central_Station_during_Ukrainian_refugee_crisis_10.jpg"
        
#     ]
# })

# fig = go.Figure(data=[
#     go.Scatter(
#         x=df['x'], 
#         y=df['y'], 
#         mode='markers',
#         marker=dict(color=df['color'])
#     )
# ])


# fig.update_traces(hoverinfo="none", hovertemplate=None)

# app = JupyterDash(_name_)

# server = app.server

# app.layout = html.Div([
#     dcc.Graph(id="graph-basic-2", figure=fig, clear_on_unhover=True),
#     dcc.Tooltip(id="graph-tooltip", direction='bottom'),
# ])


# @app.callback(
#     Output("graph-tooltip", "show"),
#     Output("graph-tooltip", "bbox"),
#     Output("graph-tooltip", "children"),
#     Output("graph-tooltip", "direction"),
    
#     Input("graph-basic-2", "hoverData"),
# )
# def display_hover(hoverData):
#     if hoverData is None:
#         return False, no_update, no_update ,no_update

#     # demo only shows the first point, but other points may also be available
#     pt = hoverData["points"][0]
#     bbox = pt["bbox"]
#     num = pt["pointNumber"]
    
    
    
#     df_row = df.iloc[num]
#     img_src = df_row['img_url']

#     children = [
#         html.Img(
#             src=img_src,
#             style={"width": "250px"},
#         ),
#         html.P("ukrain refugees"),
#     ]
    
#     y = hoverData["points"][0]['y']
#     direction = "bottom" if y > 2 else "top"
    
#     return True, bbox, children , direction

# app.run_server(mode="inline")